In [1]:
import numpy
from belashovplot import TiledPlot
from tqdm import tqdm
from copy import deepcopy

In [2]:
def norm(old:numpy.ndarray, new:numpy.ndarray):
    return numpy.max(numpy.abs(old - new))

In [3]:
def substep(d_array:numpy.ndarray, temperature:numpy.ndarray, dv:float, dz:float, Rv:float):
    d_array = deepcopy(d_array)
    
    a_array = numpy.ones(d_array.shape, dtype=complex) * (1.0 / (dv**2))
    b_array = numpy.ones(d_array.shape, dtype=complex) * (Rv*temperature - 4j/dz - 2/(dv**2))
    c_array = numpy.ones(d_array.shape, dtype=complex) * (1.0 / (dv**2))
    a_array[0] = 0
    c_array[-1] = 0

    alpha_array = numpy.zeros(d_array.shape[0]-1, dtype=complex)
    beta_array = numpy.zeros(d_array.shape[0]-1, dtype=complex)
    alpha_array[0] = -c_array[0] / b_array[0]
    beta_array[0] = d_array[0] / b_array[0]
    
    for i in range(1, d_array.shape[0]-1):
        alpha_array[i] = - c_array[i] / (a_array[i]*alpha_array[i-1] + b_array[i])
        beta_array[i] = (d_array[i] - a_array[i]*beta_array[i-1]) / (a_array[i]*alpha_array[i-1] + b_array[i])
    d_array[-1] = (d_array[-1] - a_array[-1]*beta_array[-1]) / (a_array[-1]*alpha_array[-1] + b_array[-1])
    
    for i in reversed(range(0, d_array.shape[0]-1)):
        d_array[i] = alpha_array[i]*d_array[i+1] + beta_array[i]

    return d_array

In [18]:
def substep(d_array:numpy.ndarray, temperature:numpy.ndarray, dv:float, dz:float, Rv:float):
    d_array = deepcopy(d_array)
    
    a_array = numpy.ones(d_array.shape, dtype=complex) * (1.0 / (dv**2))
    b_array = numpy.ones(d_array.shape, dtype=complex) * (Rv*temperature - 4j/dz - 2/(dv**2))
    c_array = numpy.ones(d_array.shape, dtype=complex) * (1.0 / (dv**2))
    a_array = numpy.moveaxis(a_array, len(a_array.shape)-1, 0)
    b_array = numpy.moveaxis(b_array, len(b_array.shape)-1, 0)
    c_array = numpy.moveaxis(c_array, len(c_array.shape)-1, 0)
    d_array = numpy.moveaxis(d_array, len(d_array.shape)-1, 0)
    a_array[0] = 0
    c_array[-1] = 0

    alpha_array = numpy.zeros((d_array.shape[0]-1, *d_array.shape[1:]), dtype=complex)
    beta_array = numpy.zeros((d_array.shape[0]-1, *d_array.shape[1:]), dtype=complex)
    alpha_array[0] = -c_array[0] / b_array[0]
    beta_array[0] = d_array[0] / b_array[0]
    
    for i in range(1, d_array.shape[0]-1):
        alpha_array[i] = - c_array[i] / (a_array[i]*alpha_array[i-1] + b_array[i])
        beta_array[i] = (d_array[i] - a_array[i]*beta_array[i-1]) / (a_array[i]*alpha_array[i-1] + b_array[i])
    d_array[-1] = (d_array[-1] - a_array[-1]*beta_array[-1]) / (a_array[-1]*alpha_array[-1] + b_array[-1])
    
    for i in reversed(range(0, d_array.shape[0]-1)):
        d_array[i] = alpha_array[i]*d_array[i+1] + beta_array[i]

    return numpy.moveaxis(d_array, 0, len(d_array.shape)-1)

In [19]:
def retemperature(temperature:numpy.ndarray, intensity:numpy.ndarray, dx:float):
    for m in range(temperature.shape[0]-1):
        temperature[m+1, :] = temperature[m, :] + dx*intensity[m, :]
    return temperature

In [33]:
def step(layer:numpy.ndarray, temperature:numpy.ndarray, dx:float, dy:float, dz:float, Rv:float):
    layer = deepcopy(layer)

    old_layer = deepcopy(layer)
    old_temperature = temperature + 1.0
    while norm(old_temperature, temperature) > 1.0E-3:
        # Явная по Y:
        layer = deepcopy(old_layer)
        layer0 = layer[:, :-2]
        layer1 = layer[:, 1:-1]
        layer2 = layer[:, 2:]
        d_array = deepcopy(layer)
        d_array[:, 1:-1] = 2*layer1*(1.0/(dy**2) - 2j/dz) - (layer0 + layer1)/(dy**2)
        layer = substep(d_array, temperature, dy, dz, Rv)
        # for m in range(layer.shape[0]):
        #     layer0 = layer[m, :-2]
        #     layer1 = layer[m, 1:-1]
        #     layer2 = layer[m, 2:]
        #     d_array = deepcopy(layer[m, :])
        #     d_array[1:-1] = 2*layer1*(1.0/(dy**2) - 2j/dz) - (layer0 + layer1)/(dy**2)
        #     d_array = substep(d_array, temperature[m, :], dx, dz, Rv)
        #     layer[m, :] = deepcopy(d_array)
    
        # Пересчёт температуры
        old_temperature = deepcopy(temperature)
        temperature = retemperature(temperature, numpy.abs(layer)**2, dx)
        # print(norm(old_temperature, temperature))

    old_layer = deepcopy(layer)
    old_temperature = temperature + 1.0
    while norm(old_temperature, temperature) > 1.0E-3:
        # Явная по X:
        layer = deepcopy(old_layer)
        layer0 = layer[:-2, :]
        layer1 = layer[1:-1, :]
        layer2 = layer[2:, :]
        d_array = deepcopy(layer)
        d_array[1:-1, :] = 2*layer1*(1.0/(dx**2) - 2j/dz) - (layer0 + layer1)/(dx**2)
        layer = substep(d_array, numpy.swapaxes(temperature, 0, 1), dx, dz, Rv)
        # for n in range(layer.shape[1]):
        #     layer0 = layer[:-2, n]
        #     layer1 = layer[1:-1, n]
        #     layer2 = layer[2:, n]
        #     d_array = deepcopy(layer[:, n])
        #     d_array[1:-1] = 2*layer1*(1.0/(dx**2) - 2j/dz) - (layer0 + layer1)/(dx**2)
        #     d_array = substep(d_array, temperature[:, n], dy, dz, Rv)
        #     layer[:, n] = deepcopy(d_array)
            
        # Пересчёт температуры
        old_temperature = deepcopy(temperature)
        temperature = retemperature(temperature, numpy.abs(layer)**2, dx)
        # print(norm(old_temperature, temperature))

    return layer, temperature

In [34]:
def solve(Rv:float, x0:float, x1:float, y0:float, y1:float, z0:float, z1:float, nx:int, ny:int, nz:int, f=0.4):
    x_space = numpy.linspace(x0, x1, nx)
    y_space = numpy.linspace(y0, y1, ny)
    z_space = numpy.linspace(z0, z1, nz)
    dx = x_space[1] - x_space[0]
    dy = y_space[1] - y_space[0]
    dz = z_space[1] - z_space[0]

    x_mesh, y_mesh = numpy.meshgrid(x_space, y_space)
    r2_mesh = x_mesh**2 + y_mesh**2
    layer = numpy.exp(-r2_mesh/2)*numpy.exp(0.5j*r2_mesh/f)
    
    temperature = retemperature(numpy.zeros((nx,ny)), numpy.abs(layer)**2, dx)
    
    field_map = numpy.zeros((nz,nx,ny), dtype=complex)
    temperature_map = numpy.zeros((nz,nx,ny))
    field_map[0] = deepcopy(layer)
    temperature_map[0] = deepcopy(temperature)
    for i in tqdm(range(1, nz)):
        layer, temperature = step(layer, temperature, dx, dy, dz, Rv)
        field_map[i] = deepcopy(layer)
        temperature_map[i] = deepcopy(temperature)

    return field_map, temperature_map, dx, dy, dz

In [ ]:
field, temperature, dx, dy, dz = solve(-1.0, -6.0, +6.0, -6.0, +6.0, 0.0, 0.4, 1024, 1024, 20, 0.4)

 47%|███████████████████████████████████████████████████████████████████████████████████████████████████                                                                                                              | 9/19 [00:57<01:09,  6.93s/it]

In [ ]:
field0, temperature0, _, _, _ = solve(+0.0, -6.0, +6.0, -6.0, +6.0, 0.0, 0.4, 1024, 1024, 20, 0.4)

In [ ]:
plot = TiledPlot(7.3, 9.7)
plot.title("Ветровая рефракция оптического пучка")
plot.description.row.left("Линейная $R_V=0$", 0)
plot.description.row.left("Нелинейная $R_V=-1$", 1)
plot.description.column.top("Начальная $z=0$", 0)
plot.description.column.top("Конечная $z=0.4$", 1)

axes = plot.axes.add(0,0)
axes.imshow(numpy.abs(field0[0]))

axes = plot.axes.add(1,0)
axes.imshow(numpy.abs(field0[-1]))

axes = plot.axes.add(0,1)
axes.imshow(numpy.abs(field[0]))

axes = plot.axes.add(1,1)
axes.imshow(numpy.abs(field[-1]))

plot.show()